# Extract splice junctions for AS-ALE (/ last exon spliced) events.

Due to a pretty silly move on my part, my pipeline does not output splice junctions for novel spliced last exons (nor last exons)
Given want to use SJs to quantify these events in NYGC data, need to extract using only last exon coordinates.

In pipeline, approach to define last exons:
- 3'end doesn't overlap any annotated exon
- terminal splice junction must match an annotated splice junction at the 5'end. 

Can exploit these criteria to grab SJs 'in reverse'

Strategy for novel last exons:
- Extract introns (SJs) from annotated transcripts
- find introns in which last exons are completely contained
- (w/ some form of pr.join), construct a SJ with start coordinate of annotated intron and end coordinate = start of exon coordinate
- As a sanity check, compare with AL's provided list of cryptic SJs inferred with MAJIQ - for common genes, do we get the same SJ coordinates

Strategy for annotated last exons:
- Any spliced last exon that is not captured with novel LE strategy
- pr.join with slack=1 (allow bookended intervals to overlap) annotated SJs with last exon coordinates
- Subset for exact matches at SJ 3'end/last exon 5'end
- Retain the SJs for downstream analysis


### TODOs
- Switch to clean script for generating SJs
- Create a TXT file with masked IDs (4 for now)
- For last exons not within gene bodies ('gene' entry), select the terminal introns for these transcripts, then do .nearest() to attach to last exon. Then can reconstruct SJ with existing function. 

In [1]:
import pyranges as pr
import pandas as pd
import numpy as np

In [2]:
majiq_sj = pr.read_bed("../data/Everything_minus_ferguson_liu_recent_new_cryptics.junctions.bed")
ref_gtf = pr.read_gtf("../data/reference_filtered.gtf")
le = pr.read_bed("../data/2023-07-04_papa_cryptic_spliced.last_exons.bed") # generated in motifs/ subdir, all cryptic last exon coords used for iCLIP analysis

In [3]:
# ref gtf has lots of attributes don't need, subset to minimal cols for analysis
ref_gtf = ref_gtf[["Feature", "gene_id", "transcript_id", "gene_name"]]

In [4]:
# subset le for cryptic LEs only
le_cryp = le.subset(lambda df: df.Name.str.contains("cryptic", regex=False))
ids_le_cryp = set(le_cryp.Name)
le_cryp

,Chromosome,Start,End,Name,Score,Strand
0,chr1,76871267,76871821,ENSG00000117069.15_2|ST6GALNAC5|spliced|cryptic,.,+
1,chr1,61824444,61825501,ENSG00000132849.22_1|PATJ|spliced|cryptic,.,+
2,chr1,54634687,54639192,ENSG00000162390.18_4|ACOT11|spliced|cryptic,.,+
3,chr1,245464258,245471621,ENSG00000162849.16_2|KIF26B|spliced|cryptic,.,+
4,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+
...,...,...,...,...,...,...
102,chr22,37464524,37465718,ENSG00000100060.18_2|MFNG|spliced|cryptic,.,-
103,chrX,102721363,102724864,ENSG00000198908.12_1|BHLHB9|spliced|cryptic,.,+
104,chrX,98679426,98679978,ENSG00000281566.3_1|ENSG00000281566|spliced|cr...,.,+
105,chrX,17835910,17837395,ENSG00000131831.18_1|RAI2|spliced|cryptic,.,-


In [5]:
# extract introns (SJs) for each transcript
ref_introns = ref_gtf.features.introns(by="transcript")
# store a df noting which transcript IDs share the same intron coordinates (useful if collapse later and want to track/retain)
intron2tx = ref_introns.as_df()[["Chromosome", "Start", "End", "Strand", "transcript_id"]]



In [6]:
# first extract LEs completely contained within annotated introns
le_cryp_cont = le_cryp.overlap(ref_introns, strandedness="same", how="containment")

# now overlap-join introns with LEs
ref_introns_le = ref_introns.join(le_cryp_cont, strandedness="same")
ref_introns_le

,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,Start_b,End_b,Name,Score,Strand_b
0,chr1,intron,1616614,1623430,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+
1,chr1,intron,1616614,1623430,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+
2,chr1,intron,1616614,1623430,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+
3,chr1,intron,1616614,1623430,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+
4,chr1,intron,1616614,1623430,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,chrX,intron,108221374,108310747,-,ENSG00000197565.17,ENST00000538570.5,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-
1493,chrX,intron,108221374,108310747,-,ENSG00000197565.17,ENST00000538570.5,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-
1494,chrX,intron,108221374,108310747,-,ENSG00000197565.17,ENST00000621266.4,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-
1495,chrX,intron,108221374,108310747,-,ENSG00000197565.17,ENST00000621266.4,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-


In [7]:
# If + strand
# Start coord = intron start (i.e. 'Start' column), End coord = exon start (i.e. 'Start_b' column)
# If - strand
# Start coord = Exon start (i.e. 'End_b' column), End coord = intron start (i.e. 'End' column)

def _df_intron_exon_to_sj(df: pd.DataFrame) -> pd.DataFrame:
    '''Convert joined intron + overlapping exon df of PyRanges object to splice junction coordinates (the first bin between the two intervals)

    If + strand
    Start coord = intron start (i.e. 'Start' column), End coord = exon start (i.e. 'Start_b' column)
    If - strand
    Start coord = Exon start (i.e. 'End_b' column), End coord = intron start (i.e. 'End' column)
    
    Parameters
    ----------
    df : pd.DataFrame
        _description_

    Returns
    -------
    pd.DataFrame
        _description_

    Raises
    ------
    Exception
        Strand column of df doesn't contain all '+' or '-'
    '''

    df[["Start_intron", "End_intron"]] = df[["Start", "End"]]

    if (df.Strand == "+").all():
        df["End"] = df["Start_b"]

    elif (df.Strand == "-").all():
        df["Start"] = df["End_b"]

    else:
        raise Exception("Strand column must only contain all '+' or '-'")
    
    return df


le_sj = ref_introns_le.apply(_df_intron_exon_to_sj)
le_sj

,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,Start_b,End_b,Name,Score,Strand_b,Start_intron,End_intron
0,chr1,intron,1616614,1616614,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,1616614,1623430
1,chr1,intron,1616614,1616614,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,1616614,1623430
2,chr1,intron,1616614,1616614,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,1616614,1623430
3,chr1,intron,1616614,1616614,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,1616614,1623430
4,chr1,intron,1616614,1616614,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,1616614,1623430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,chrX,intron,108269152,108310747,-,ENSG00000197565.17,ENST00000538570.5,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-,108221374,108310747
1493,chrX,intron,108269152,108310747,-,ENSG00000197565.17,ENST00000538570.5,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-,108221374,108310747
1494,chrX,intron,108269152,108310747,-,ENSG00000197565.17,ENST00000621266.4,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-,108221374,108310747
1495,chrX,intron,108269152,108310747,-,ENSG00000197565.17,ENST00000621266.4,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-,108221374,108310747


In [8]:
le_sj.subset(lambda df: df.End - df.Start == 0).drop_duplicate_positions() # perhaps these two are annotated in some way... or bleedthroughs with annotated internal exon chunk removed

,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,Start_b,End_b,Name,Score,Strand_b,Start_intron,End_intron
0,chr1,intron,1616614,1616614,+,ENSG00000197530.13,ENST00000355826.10,MIB2,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,1616614,1623430
1,chr1,intron,20776596,20776596,-,ENSG00000127483.19,ENST00000312239.10,HP1BP3,20775449,20776596,ENSG00000127483.19_1|HP1BP3|spliced|cryptic,.,-,20773610,20776596


In [9]:
# remove 0 length SJs for now as likely some annotation issue - check back at end after mined annotated SJs
le_sj = le_sj.subset(lambda df: df.Start != df.End)
le_sj

,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,Start_b,End_b,Name,Score,Strand_b,Start_intron,End_intron
0,chr1,intron,245419745,245464258,+,ENSG00000162849.16,ENST00000407071.7,KIF26B,245464258,245471621,ENSG00000162849.16_2|KIF26B|spliced|cryptic,.,+,245419745,245540766
1,chr1,intron,61823079,61824444,+,ENSG00000132849.22,ENST00000459752.5,PATJ,61824444,61825501,ENSG00000132849.22_1|PATJ|spliced|cryptic,.,+,61823079,61827421
2,chr1,intron,61823079,61824444,+,ENSG00000132849.22,ENST00000459752.5,PATJ,61824444,61825501,ENSG00000132849.22_1|PATJ|spliced|cryptic,.,+,61823079,61827421
3,chr1,intron,61823079,61824444,+,ENSG00000132849.22,ENST00000459752.5,PATJ,61824444,61825501,ENSG00000132849.22_1|PATJ|spliced|cryptic,.,+,61823079,61827421
4,chr1,intron,76868742,76871267,+,ENSG00000117069.15,ENST00000477717.6,ST6GALNAC5,76871267,76871821,ENSG00000117069.15_2|ST6GALNAC5|spliced|cryptic,.,+,76868742,77044203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,chrX,intron,108269152,108310747,-,ENSG00000197565.17,ENST00000538570.5,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-,108221374,108310747
1359,chrX,intron,108269152,108310747,-,ENSG00000197565.17,ENST00000538570.5,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-,108221374,108310747
1360,chrX,intron,108269152,108310747,-,ENSG00000197565.17,ENST00000621266.4,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-,108221374,108310747
1361,chrX,intron,108269152,108310747,-,ENSG00000197565.17,ENST00000621266.4,COL4A6,108267641,108269152,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.,-,108221374,108310747


In [10]:
# finally generate a cleaned BED-ready file of SJs
le_sj_clean_nov = le_sj[["Name"]]
le_sj_clean_nov.Score = "."
le_sj_clean_nov

,Chromosome,Start,End,Strand,Name,Score
0,chr1,245419745,245464258,+,ENSG00000162849.16_2|KIF26B|spliced|cryptic,.
1,chr1,61823079,61824444,+,ENSG00000132849.22_1|PATJ|spliced|cryptic,.
2,chr1,61823079,61824444,+,ENSG00000132849.22_1|PATJ|spliced|cryptic,.
3,chr1,61823079,61824444,+,ENSG00000132849.22_1|PATJ|spliced|cryptic,.
4,chr1,76868742,76871267,+,ENSG00000117069.15_2|ST6GALNAC5|spliced|cryptic,.
...,...,...,...,...,...,...
1358,chrX,108269152,108310747,-,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.
1359,chrX,108269152,108310747,-,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.
1360,chrX,108269152,108310747,-,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.
1361,chrX,108269152,108310747,-,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.


In [11]:
# now to get SJs for annotated ALEs
# overlap-join SJs and LEs, allowing bookended/directly adjacent to overlap
# subset for direct match at intron 3'end, last exon 5'EncodingWarning
le_sj_ref = (ref_introns.join(le_cryp, strandedness="same", slack=1)
 .subset(lambda df: ((df.Strand == "+") & (df["End"] == df["Start_b"])) |
         ((df.Strand == "-") & (df["Start"] == df["End_b"]))
         )
         .drop_duplicate_positions()
 )

le_sj_ref

,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,Start_b,End_b,Name,Score,Strand_b
0,chr1,intron,54630886,54634687,+,ENSG00000162390.18,ENST00000371316.3,ACOT11,54634687,54639192,ENSG00000162390.18_4|ACOT11|spliced|cryptic,.,+
1,chr1,intron,112542090,112550600,-,ENSG00000007341.19,ENST00000343210.11,ST7L,112540178,112542090,ENSG00000007341.19_4|ST7L|spliced|cryptic,.,-
2,chr1,intron,112542090,112555867,-,ENSG00000007341.19,ENST00000360743.8,ST7L,112540178,112542090,ENSG00000007341.19_4|ST7L|spliced|cryptic,.,-
3,chr1,intron,33592489,33600864,-,ENSG00000121904.19,ENST00000373380.5,CSMD2,33591806,33592489,ENSG00000121904.19_3|CSMD2|spliced|cryptic,.,-
4,chr1,intron,163305313,163306232,-,ENSG00000232995.7,ENST00000439699.1,ENSG00000232995,163304210,163305313,ENSG00000232995.7_2|ENSG00000232995|spliced|cr...,.,-
5,chr2,intron,205766818,205776245,+,ENSG00000118257.17,ENST00000272849.7,NRP2,205776245,205779714,ENSG00000118257.17_5|NRP2|spliced|cryptic,.,+
6,chr2,intron,205766803,205776245,+,ENSG00000118257.17,ENST00000357118.8,NRP2,205776245,205779714,ENSG00000118257.17_5|NRP2|spliced|cryptic,.,+
7,chr2,intron,3457693,3457794,+,ENSG00000171853.16,ENST00000441099.5,TRAPPC12,3457794,3458181,ENSG00000171853.16_4|TRAPPC12|spliced|cryptic,.,+
8,chr3,intron,3129188,3129918,+,ENSG00000072756.17,ENST00000339437.11,TRNT1,3129918,3130679,ENSG00000072756.17_1|TRNT1|spliced|cryptic,.,+
9,chr3,intron,114339426,114350273,-,ENSG00000181722.17,ENST00000357258.8,ZBTB20,114314500,114339426,ENSG00000181722.17_5|ZBTB20|spliced|cryptic,.,-


In [12]:
# finally generate a cleaned BED-ready file of ref SJs
le_sj_clean_ref = le_sj_ref[["Name"]]
le_sj_clean_ref.Score = "."
le_sj_clean_ref

,Chromosome,Start,End,Strand,Name,Score
0,chr1,54630886,54634687,+,ENSG00000162390.18_4|ACOT11|spliced|cryptic,.
1,chr1,112542090,112550600,-,ENSG00000007341.19_4|ST7L|spliced|cryptic,.
2,chr1,112542090,112555867,-,ENSG00000007341.19_4|ST7L|spliced|cryptic,.
3,chr1,33592489,33600864,-,ENSG00000121904.19_3|CSMD2|spliced|cryptic,.
4,chr1,163305313,163306232,-,ENSG00000232995.7_2|ENSG00000232995|spliced|cr...,.
5,chr2,205766818,205776245,+,ENSG00000118257.17_5|NRP2|spliced|cryptic,.
6,chr2,205766803,205776245,+,ENSG00000118257.17_5|NRP2|spliced|cryptic,.
7,chr2,3457693,3457794,+,ENSG00000171853.16_4|TRAPPC12|spliced|cryptic,.
8,chr3,3129188,3129918,+,ENSG00000072756.17_1|TRNT1|spliced|cryptic,.
9,chr3,114339426,114350273,-,ENSG00000181722.17_5|ZBTB20|spliced|cryptic,.


In [13]:
# now just need to check that all les are represented
le_sj_clean = pr.concat([le_sj_clean_nov.drop_duplicate_positions(), le_sj_clean_ref])
le_sj_clean

,Chromosome,Start,End,Strand,Name,Score
0,chr1,245419745,245464258,+,ENSG00000162849.16_2|KIF26B|spliced|cryptic,.
1,chr1,61823079,61824444,+,ENSG00000132849.22_1|PATJ|spliced|cryptic,.
2,chr1,76868742,76871267,+,ENSG00000117069.15_2|ST6GALNAC5|spliced|cryptic,.
3,chr1,54630886,54634687,+,ENSG00000162390.18_4|ACOT11|spliced|cryptic,.
4,chr1,243613034,243613670,-,ENSG00000117020.19_2|AKT3|spliced|cryptic,.
...,...,...,...,...,...,...
153,chrX,98644574,98679426,+,ENSG00000281566.3_1|ENSG00000281566|spliced|cr...,.
154,chrX,17837395,17837536,-,ENSG00000131831.18_1|RAI2|spliced|cryptic,.
155,chrX,108269152,108310747,-,ENSG00000197565.17_1|COL4A6|spliced|cryptic,.
156,chrX,17837395,17861097,-,ENSG00000131831.18_1|RAI2|spliced|cryptic,.


In [14]:
# check overlap with AL's cryptics BED
# get gene names from le_sj_id_clean
# Name field = ENSG00000162849.16_2|KIF26B|spliced|cryptic	
le_cryp_gn = set(le_sj_clean.Name.str.split("|", expand=True)[1])

# Name field = PTPRU|novel_donor|2	
majiq_sj_gn = set(majiq_sj.Name.str.split("|", expand=True)[0])

# get matching genes
le_majiq_m_gn = le_cryp_gn.intersection(majiq_sj_gn)

print(f"Fraction of cryptics where gene has SJ found by MAJIQ - {len(le_majiq_m_gn) / len(le_cryp_gn)}")
#


Fraction of cryptics where gene has SJ found by MAJIQ - 0.7526881720430108


In [15]:
# extract gene_name as column
le_sj_clean = le_sj_clean.assign("gene_name", lambda df: df.Name.str.split("|", expand=True)[1])
majiq_sj = majiq_sj.assign("gene_name", lambda df: df.Name.str.split("|", expand=True)[0])

# overlap-join le inferred SJs with MAJIQ SJs 
sj_le_majiq = (le_sj_clean.subset(lambda df: df.gene_name.isin(le_majiq_m_gn))
 .join(majiq_sj.subset(lambda df: df.gene_name.isin(le_majiq_m_gn)),
       how="left",
       strandedness="same"
        )
 )

# calculate difference between 3' coordinates
(sj_le_majiq.assign("end3_diff",
                    lambda df: abs(df.End_b - df.End) if (df.Strand == "+").all() else abs(df.Start_b - df.Start))
                    .apply(lambda df: df.loc[df.groupby("Name")["end3_diff"].idxmin(), :])
                    .end3_diff.describe(percentiles=[0.1 * i for i in range(0,11,1)])
)


count    7.400000e+01
mean     1.789569e+07
std      4.010586e+07
min      0.000000e+00
0%       0.000000e+00
10%      0.000000e+00
20%      0.000000e+00
30%      1.000000e+00
40%      1.000000e+00
50%      1.000000e+00
60%      1.689400e+03
70%      1.410880e+04
80%      2.557911e+07
90%      7.430823e+07
100%     2.057762e+08
max      2.057762e+08
Name: end3_diff, dtype: float64

In [16]:
(sj_le_majiq.assign("end5_diff",
                    lambda df: abs(df.Start_b - df.Start) if (df.Strand == "+").all() else abs(df.End_b - df.End))
                    .apply(lambda df: df.loc[df.groupby("Name")["end5_diff"].idxmin(), :]) # select smallest per gene (so prioritises match)
                    .end5_diff.describe(percentiles=[0.1 * i for i in range(0,11,1)])
)

count    7.400000e+01
mean     1.789371e+07
std      4.010648e+07
min      0.000000e+00
0%       0.000000e+00
10%      0.000000e+00
20%      0.000000e+00
30%      0.000000e+00
40%      1.000000e+00
50%      1.000000e+00
60%      1.000000e+00
70%      2.867100e+03
80%      2.557979e+07
90%      7.430702e+07
100%     2.057668e+08
max      2.057668e+08
Name: end5_diff, dtype: float64

20-30 % are exactly matching at 3'end (start of last exon), but further 20-30 % differ by 1 nucleotide... One off error?

5'end matching is slightly better...

Let's IGV browse a few 

In [17]:
# Add 5' and 3' difference to df
sj_le_majiq = (sj_le_majiq.assign("end3_diff",
                                                      lambda df: abs(df.End_b - df.End) if (df.Strand == "+").all() else abs(df.Start_b - df.Start))
                    .assign("end5_diff",
                                                lambda df: abs(df.Start_b - df.Start) if (df.Strand == "+").all() else abs(df.End_b - df.End))
                    )

In [18]:
sj_le_majiq.subset(lambda df: df.end3_diff + df.end5_diff <= 1).subset(lambda df: df.gene_name.isin(["STMN2", "ARHGAP32", "SYNJ2", "GLE1", "ONECUT1"])).drop_duplicate_positions()

,Chromosome,Start,End,Strand,Name,Score,gene_name,Start_b,End_b,Name_b,Score_b,Strand_b,gene_name_b,end3_diff,end5_diff
0,chr6,158017290,158019983,+,ENSG00000078269.15_1|SYNJ2|spliced|cryptic,.,SYNJ2,158017290,158019984,SYNJ2|novel_acceptor|5,0,+,SYNJ2,1,0
1,chr8,79611214,79616821,+,ENSG00000104435.14_1|STMN2|spliced|cryptic,.,STMN2,79611214,79616822,STMN2|novel_acceptor|23,0,+,STMN2,1,0
2,chr8,79611791,79616821,+,ENSG00000104435.14_1|STMN2|spliced|cryptic,.,STMN2,79611791,79616822,STMN2|novel_acceptor|1,0,+,STMN2,1,0
3,chr11,128992046,128998318,-,ENSG00000134909.19_1|ARHGAP32|spliced|cryptic,.,ARHGAP32,128992046,128998319,ARHGAP32|novel_acceptor|21,0,-,ARHGAP32,0,1
4,chr15,52777690,52788779,-,ENSG00000169856.9_1|ONECUT1|spliced|cryptic,.,ONECUT1,52777690,52788780,ONECUT1|novel_acceptor|5,0,-,ONECUT1,0,1


In [19]:
# matches seem to have a difference of 1 at either 5'/3' end
# I think this is due to End coordinates being systematically shifted
# If so, then when subset for matches the one-off difference should be perfectly split by strand
# (+ strand - 3'end = End coordinate, - strand - 5'end = End coordinate)
(sj_le_majiq.subset(lambda df: df.end3_diff + df.end5_diff <= 1)
 .drop_duplicate_positions()
 .assign("end_diff", lambda df: abs(df.End_b - df.End))
 .as_df()[["Strand", "Name", "end_diff"]]
 .groupby("Strand")
 .end_diff.describe(percentiles=[0.1 * i for i in range(0,11,1)])
 )

,count,mean,std,min,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%,max
Strand,,,,,,,,,,,,,,,,
+,21.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
-,20.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [20]:
# output current SJ BED just so I can vis in IGV
# le_sj_clean.drop("gene_name").to_bed("../processed/2023-09-01_putative_as_cle_sjs.bed")

In [21]:
# # potentially a strand & End coordinate problem? AL's BED is always +1?
# (sj_le_majiq.assign("end_diff", lambda df: abs(df.End_b - df.End))
#  .apply(lambda df: df.loc[df.groupby("Name")["end_diff"].idxmin(), :]) # select smallest per gene (so prioritises match)
#  .as_df()
#  [["Name", "end_diff","Strand"]]
#  .groupby("Strand")
#  .end_diff.describe(percentiles=[0.1 * i for i in range(0,11,1)])
# )

In [33]:
# which le_ids do not have a SJ?
missing_le_ids = ids_le_cryp.difference(set(le_sj_clean.Name))
print(len(missing_le_ids))
missing_le_ids

10


{'ENSG00000002746.15_3|HECW1|spliced|cryptic',
 'ENSG00000100060.18_2|MFNG|spliced|cryptic',
 'ENSG00000111665.12_4|CDCA3|spliced|cryptic',
 'ENSG00000127483.19_1|HP1BP3|spliced|cryptic',
 'ENSG00000151692.15_6|RNF144A|spliced|cryptic',
 'ENSG00000151692.15_7|RNF144A|spliced|cryptic',
 'ENSG00000183570.17_3|PCBP3|spliced|cryptic',
 'ENSG00000184347.15_7|SLIT3|spliced|cryptic',
 'ENSG00000197530.13_1|MIB2|spliced|cryptic',
 'ENSG00000216895.9_4|ENSG00000216895|spliced|cryptic'}

In [49]:
# which ones have an overlapping reference SJ/intron?
le_cryp_missing_sj_overlap = (le_cryp.subset(lambda df: df.Name.isin(missing_le_ids))
.join(ref_introns, how="left", strandedness="same", slack=1)
.apply(lambda df: df.drop_duplicates(subset=["Start", "End", "Name", "Start_b", "End_b"]))
.assign("end3_diff",
        lambda df: abs(df.End_b - df.End) if (df.Strand == "+").all() else abs(df.Start_b - df.Start))
.assign("end5_diff",
        lambda df: abs(df.Start_b - df.Start) if (df.Strand == "+").all() else abs(df.End_b - df.End))
)


le_cryp_missing_sj_overlap

,Chromosome,Start,End,Name,Score,Strand,Feature,Start_b,End_b,Strand_b,gene_id,transcript_id,gene_name,end3_diff,end5_diff
0,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,intron,1616614,1623430,+,ENSG00000197530.13,ENST00000355826.10,MIB2,4220,0
1,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,intron,1616614,1623388,+,ENSG00000197530.13,ENST00000489635.5,MIB2,4178,0
2,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,intron,1616614,1617077,+,ENSG00000197530.13,ENST00000463492.1,MIB2,2133,0
3,chr1,20775449,20776596,ENSG00000127483.19_1|HP1BP3|spliced|cryptic,.,-,intron,20773610,20776596,-,ENSG00000127483.19,ENST00000312239.10,HP1BP3,1839,0
4,chr1,20775449,20776596,ENSG00000127483.19_1|HP1BP3|spliced|cryptic,.,-,intron,20776005,20776596,-,ENSG00000127483.19,ENST00000375000.5,HP1BP3,556,0
5,chr2,7258415,7261502,ENSG00000151692.15_6|RNF144A|spliced|cryptic,.,+,-1,-1,-1,+,-1,-1,-1,7261503,7258416
6,chr2,7286015,7287851,ENSG00000151692.15_7|RNF144A|spliced|cryptic,.,+,-1,-1,-1,+,-1,-1,-1,7287852,7286016
7,chr5,168585496,168586008,ENSG00000184347.15_7|SLIT3|spliced|cryptic,.,-,-1,-1,-1,-,-1,-1,-1,168585497,168586009
8,chr7,43444214,43446033,ENSG00000002746.15_3|HECW1|spliced|cryptic,.,+,intron,43442629,43444217,+,ENSG00000002746.15,ENST00000395891.7,HECW1,1816,1585
9,chr7,43444214,43446033,ENSG00000002746.15_3|HECW1|spliced|cryptic,.,+,intron,43445570,43456296,+,ENSG00000002746.15,ENST00000453890.5,HECW1,10263,1356


In [48]:
# Only a few match/overlap known SJs. In each case, the cryptic is further downstream of the known SJ. i.e. these are likely distal to known last exon (so can consider valid)
# double check: do 3'ends overlap with known/reference exons?
(le_cryp_missing_sj_overlap
 .subset(lambda df: df.end5_diff.eq(0))
 .three_end()
 .drop(like="_b$")
 .drop("Feature")
 .join(ref_gtf.subset(lambda df: df.Feature == "exon"), how="left", strandedness="same")
)



,Chromosome,Start,End,Name,Score,Strand,gene_id,transcript_id,gene_name,end3_diff,end5_diff,Feature,Start_b,End_b,Strand_b,gene_id_b,transcript_id_b,gene_name_b
0,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000355826.10,MIB2,4220,0,exon,1616507,1619210,+,ENSG00000197530.13,ENST00000477990.1,MIB2
1,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000355826.10,MIB2,4220,0,exon,1616507,1617898,+,ENSG00000197530.13,ENST00000507082.1,MIB2
2,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000355826.10,MIB2,4220,0,exon,1617077,1617323,+,ENSG00000197530.13,ENST00000463492.1,MIB2
3,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000489635.5,MIB2,4178,0,exon,1616507,1619210,+,ENSG00000197530.13,ENST00000477990.1,MIB2
4,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000489635.5,MIB2,4178,0,exon,1616507,1617898,+,ENSG00000197530.13,ENST00000507082.1,MIB2
5,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000489635.5,MIB2,4178,0,exon,1617077,1617323,+,ENSG00000197530.13,ENST00000463492.1,MIB2
6,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000463492.1,MIB2,2133,0,exon,1616507,1619210,+,ENSG00000197530.13,ENST00000477990.1,MIB2
7,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000463492.1,MIB2,2133,0,exon,1616507,1617898,+,ENSG00000197530.13,ENST00000507082.1,MIB2
8,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000463492.1,MIB2,2133,0,exon,1617077,1617323,+,ENSG00000197530.13,ENST00000463492.1,MIB2
9,chr1,20775449,20776596,ENSG00000127483.19_1|HP1BP3|spliced|cryptic,.,-,ENSG00000127483.19,ENST00000312239.10,HP1BP3,1839,0,exon,20775449,20776750,-,ENSG00000127483.19,ENST00000487117.1,HP1BP3


In [51]:
# what about overlap with known exons?
(le_cryp_missing_sj_overlap
 .subset(lambda df: df.end5_diff.eq(0))
 .drop(like="_b$")
 .drop("Feature")
 .join(ref_gtf.subset(lambda df: df.Feature == "exon"), how="left", strandedness="same")
 .apply(lambda df: df.drop_duplicates(subset=["Start", "End", "Name", "Start_b", "End_b"]))
)

,Chromosome,Start,End,Name,Score,Strand,gene_id,transcript_id,gene_name,end3_diff,end5_diff,Feature,Start_b,End_b,Strand_b,gene_id_b,transcript_id_b,gene_name_b
0,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000355826.10,MIB2,4220,0,exon,1616507,1619210,+,ENSG00000197530.13,ENST00000477990.1,MIB2
1,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000355826.10,MIB2,4220,0,exon,1616507,1617898,+,ENSG00000197530.13,ENST00000507082.1,MIB2
2,chr1,1616614,1619210,ENSG00000197530.13_1|MIB2|spliced|cryptic,.,+,ENSG00000197530.13,ENST00000355826.10,MIB2,4220,0,exon,1617077,1617323,+,ENSG00000197530.13,ENST00000463492.1,MIB2
3,chr1,20775449,20776596,ENSG00000127483.19_1|HP1BP3|spliced|cryptic,.,-,ENSG00000127483.19,ENST00000312239.10,HP1BP3,1839,0,exon,20775449,20776750,-,ENSG00000127483.19,ENST00000487117.1,HP1BP3
4,chr1,20775449,20776596,ENSG00000127483.19_1|HP1BP3|spliced|cryptic,.,-,ENSG00000127483.19,ENST00000312239.10,HP1BP3,1839,0,exon,20775654,20776005,-,ENSG00000127483.19,ENST00000375000.5,HP1BP3
5,chr12,6844792,6846724,ENSG00000111665.12_4|CDCA3|spliced|cryptic,.,-,ENSG00000111665.12,ENST00000603043.1,CDCA3,1575,0,exon,6844792,6846903,-,ENSG00000111665.12,ENST00000422785.7,CDCA3
6,chr12,6844792,6846724,ENSG00000111665.12_4|CDCA3|spliced|cryptic,.,-,ENSG00000111665.12,ENST00000603043.1,CDCA3,1575,0,exon,6844798,6846903,-,ENSG00000111665.12,ENST00000604599.1,CDCA3
7,chr12,6844792,6846724,ENSG00000111665.12_4|CDCA3|spliced|cryptic,.,-,ENSG00000111665.12,ENST00000603043.1,CDCA3,1575,0,exon,6844858,6846367,-,ENSG00000111665.12,ENST00000603043.1,CDCA3


In [54]:
# IGV browsing
le_cryp.subset(lambda df: df.Name.isin(missing_le_ids)).to_bed("../processed/2023-09-04_papa_as_ale_cryptic.missing.last_exon.bed")


3 genes - 3'end matches a known exon 3'end but annotation-novel 5'ss
- MIB2 - 1 annotated intron that completely contains the last exon. Intron also starts at same pos as last exon. 
- HP1BP3 - 5'end internal to 1 exon, downstream of another (which has 3'end within cryptic last exon). Intron also starts at same pos as last exon. 
- CDCA3 -exon start distinct to two tx but upstream of another. 3'end exactly matches 1 annotated tx.  Intron also starts at same pos as last exon.

all some kind of weird 3'UTR intron retention + polyadenylation event? or some kind of artefact of making chunks?

- 'ENSG00000002746.15_3|HECW1|spliced|cryptic' - overlaps but not fully contained within an intron. More like a bleedthrough than an AS-ALE
'ENSG00000100060.18_2|MFNG|spliced|cryptic', - no overlapping intron, downstream of annotated last exon
'ENSG00000111665.12_4|CDCA3|spliced|cryptic' - not fully contained within an intron, 3'ss is internal to annotated last exon. 3'end matches annotated 3'end. With respect to annot there is a spliced 3'UTR intron that is starts where cryptic ends. as there are multiple ALEs, my guess is that it has been lumped in with other overlapping isoforms at this site. Is it region with non-last exon subtracted?
'ENSG00000127483.19_1|HP1BP3|spliced|cryptic' - exon described as bleedthrough, but seems to have overlapping internal exons subtracted?
'ENSG00000151692.15_6|RNF144A|spliced|cryptic' - no overlapping intron, downstream of annotated last exon
'ENSG00000151692.15_7|RNF144A|spliced|cryptic' - no overlapping intron, downstream of annotated last exon
'ENSG00000183570.17_3|PCBP3|spliced|cryptic' - Looks more like a standard cryptic anyway. But very complex splicing patterns going on. Fully contained within annotated intron, but unclear how there is a spanning SJ...
'ENSG00000184347.15_7|SLIT3|spliced|cryptic' - no overlapping intron, downstream of annotated last exon
'ENSG00000197530.13_1|MIB2|spliced|cryptic' - better described as bleedthrough with overlapping internal exons subtracted
'ENSG00000216895.9_4|ENSG00000216895|spliced|cryptic' - possibly downstream of annotated last exon (overlapping introns seem to belong to poorly supported transcripts)

Summary: 
- 5 / 10 are downstream of annotated last exon (i.e. no overlapping/contained intron)
- 3 / 10 that are possibly bleedthroughs/have internal exon subtracted? Or that a representative 5'end has been selected that 'isn't representative' i.e. 1 / x last exons with overlapping sequence, pick an internal one (double check iCLIP code)
- HECW1 - not fully contained within an intron
- PCBP3 - nothing clear cut. Again possibly an issue with a non-representative 5'end being selected. 

HECW1 seems to have been misannotated anyway, so can ignore that

Solution:
- Allow LE to be downstream of gene, in those cases find the most distal annotated intron, then construct SJ as 5'end of last annotated intron to LE start
- MIB2, HP1BP3 & CDCA3 - understand why & how internal exon subtracted region has been selected (back to iCLIP regions filtering?).




### Output BEDs to file

Above, I've noticed that many matching splice junctions (e.g. STMN2, ONECUT1 etc.) are shifted by 1nt in the End coordinate. According to AL this is a requirement of her bedops_parse_star_junctions pipeline in order to match with STARs .sj.out.tab convention for reporting SJs. So will output:
1. BED file with unmodified SJs (all juncs)
2. BED file with End + 1 SJs (all juncs)
3. BED file with missing LEs/lacking a SJ
4. 1 & 2 with only those matching majiq junctions



In [31]:
# Get a set of cryptics thay have an exactly matching MAJIQ SJ.
# Note: Start coordinate should be identical, End coordinate will be shifted by +1 if directly match

# df of Name | Name_b (PAPA ID | MAJIQ ID)
papa2majiq_sj = (sj_le_majiq.subset(lambda df: (df.Start == df.Start_b) & (df.End_b - df.End == 1))
                 .as_df()
                 [["Name", "Name_b"]]
                 .drop_duplicates()
                 .rename(columns={"Name": "papa_name", "Name_b": "majiq_name"})
                 )

print(papa2majiq_sj.papa_name.nunique())
papa2majiq_sj


40


,papa_name,majiq_name
0,ENSG00000162849.16_2|KIF26B|spliced|cryptic,KIF26B|novel_acceptor|4
4,ENSG00000132849.22_1|PATJ|spliced|cryptic,PATJ|novel_acceptor|8
13,ENSG00000117020.19_2|AKT3|spliced|cryptic,AKT3|annotated|2
15,ENSG00000115839.19_2|RAB3GAP1|spliced|cryptic,RAB3GAP1|annotated|1
16,ENSG00000171853.16_4|TRAPPC12|spliced|cryptic,TRAPPC12|annotated|14
46,ENSG00000115392.13_1|FANCL|spliced|cryptic,FANCL|novel_acceptor|7
55,ENSG00000171094.18_1|ALK|spliced|cryptic,ALK|novel_acceptor|4
60,ENSG00000244405.8_3|ETV5|spliced|cryptic,ETV5|novel_acceptor|9
71,ENSG00000114541.15_3|FRMD4B|spliced|cryptic,FRMD4B|annotated|4
75,ENSG00000145349.18_3|CAMK2D|spliced|cryptic,CAMK2D|novel_acceptor|1


In [32]:

le_sj_clean = le_sj_clean.drop("gene_name")
# unmodified sjs]
le_sj_clean.to_bed("../processed/2023-09-04_papa_as_ale_cryptic.all.unmodified.junctions.bed")
# add 1 to End coord
le_sj_clean.assign("End", lambda df: df.End.add(1)).to_bed("../processed/2023-09-04_papa_as_ale_cryptic.all.end_shift.junctions.bed")


# majiq matched only
le_sj_clean.subset(lambda df: df.Name.isin(papa2majiq_sj.papa_name)).to_bed("../processed/2023-09-04_papa_as_ale_cryptic.majiq_match.unmodified.junctions.bed")
le_sj_clean.subset(lambda df: df.Name.isin(papa2majiq_sj.papa_name)).assign("End", lambda df: df.End.add(1)).to_bed("../processed/2023-09-04_papa_as_ale_cryptic.majiq_match.end_shift.junctions.bed")